## Решение задач с ограничениями

In [ ]:
from simpleai.search import CspProblem, backtrack, \
        min_conflicts, MOST_CONSTRAINED_VARIABLE, \
        HIGHEST_DEGREE_VARIABLE, LEAST_CONSTRAINING_VALUE

# Определение функций-ограничений

def constraint_unique(variables, values):
    """Ограничение уникальности: все значения должны быть разными"""
    return len(values) == len(set(values))  # Проверяем, что нет дубликатов

def constraint_bigger(variables, values):
    """Ограничение сравнения: первое значение должно быть больше второго"""
    return values[0] > values[1]  # Том > Анны

def constraint_odd_even(variables, values):
    """Ограничение четности: если первое число четное, второе должно быть нечетным, и наоборот"""
    if values[0] % 2 == 0:
        return values[1] % 2 == 1  # Джон четный → Патрисия нечетная
    else:
        return values[1] % 2 == 0  # Джон нечетный → Патрисия четная

if __name__ == '__main__':
    # Определение переменных задачи
    variables = ('Джон', 'Анна', 'Том', 'Патрисия')

    # Домены значений для каждой переменной (возможные значения)
    domains = {
        'Джон': [1, 2, 3, 4],
        'Анна': [1, 3, 5],        # Только нечетные, кроме 5
        'Том': [2, 4, 3],         # В основном четные + 3
        'Патрисия': [2, 3, 4, 5], # Все доступные значения
    }

    # Определение ограничений между переменными
    constraints = [
        # Тройное ограничение: Джон, Анна и Том должны иметь разные значения
        (('Джон', 'Анна', 'Том'), constraint_unique), 
        # Парное ограничение: значение Тома должно быть больше значения Анны
        (('Том', 'Анна'), constraint_bigger),          
        # Парное ограничение: Джон и Патрисия должны иметь разную четность
        (('Джон', 'Патрисия'), constraint_odd_even),   
    ]

    # Создание объекта задачи удовлетворения ограничений (CSP)
    problem = CspProblem(variables, domains, constraints)

    # Решение задачи различными методами и вывод результатов

    # 1. Стандартный backtracking (обратный отсчет) без эвристик
    print('\nРешения:\n\nОбычный поиск:', backtrack(problem))
    
    # 2. Backtracking с эвристикой "наиболее ограниченная переменная"
    # (выбирается переменная с наименьшим количеством доступных значений)
    print('\nНаиболее ограниченная переменная:', backtrack(problem, 
            variable_heuristic=MOST_CONSTRAINED_VARIABLE))
    
    # 3. Backtracking с эвристикой "переменная с наибольшей степенью"
    # (выбирается переменная, участвующая в наибольшем количестве ограничений)
    print('\nПеременная с наибольшей степенью:', backtrack(problem, 
            variable_heuristic=HIGHEST_DEGREE_VARIABLE))
    
    # 4. Backtracking с эвристикой "наименее ограничивающее значение"
    # (выбирается значение, которое оставляет максимальную свободу другим переменным)
    print('\nНаименее ограничивающее значение:', backtrack(problem, 
            value_heuristic=LEAST_CONSTRAINING_VALUE))
    
    # 5. Комбинация: наиболее ограниченная переменная + наименее ограничивающее значение
    print('\nНаиболее ограниченная переменная и наименее ограничивающее значение:', 
            backtrack(problem, variable_heuristic=MOST_CONSTRAINED_VARIABLE, 
            value_heuristic=LEAST_CONSTRAINING_VALUE))
    
    # 6. Комбинация: переменная с наибольшей степенью + наименее ограничивающее значение
    print('\nНаибольшая степень и наименее ограничивающее значение:', 
            backtrack(problem, variable_heuristic=HIGHEST_DEGREE_VARIABLE, 
            value_heuristic=LEAST_CONSTRAINING_VALUE))
    
    # 7. Алгоритм минимальных конфликтов (min-conflicts)
    # (локальный поиск, который пытается минимизировать количество конфликтов)
    print('\nМинимальные конфликты:', min_conflicts(problem))

In [ ]:
import argparse  # Импорт модуля для обработки аргументов командной строки
import simpleai.search as ss  # Импорт библиотеки для поисковых алгоритмов

def build_arg_parser():
    """Создает и настраивает парсер аргументов командной строки"""
    parser = argparse.ArgumentParser(description='Создаёт целевую строку с помощью жадного алгоритма')
    # Обязательный аргумент: целевая строка, которую нужно получить
    parser.add_argument("--input-string", dest="input_string", required=True,
            help="Целевая строка")
    # Необязательный аргумент: начальное состояние (по умолчанию - пустая строка)
    parser.add_argument("--initial-state", dest="initial_state", required=False,
            default='', help="Начальное состояние для поиска")
    return parser

class CustomProblem(ss.SearchProblem):
    """Пользовательский класс задачи поиска для построения целевой строки"""
    
    def set_target(self, target_string):
        """Устанавливает целевую строку для задачи"""
        self.target_string = target_string

    def actions(self, cur_state):
        """
        Возвращает список возможных действий из текущего состояния.
        Действие - это добавление одного символа.
        """
        # Если текущая строка короче целевой, можно добавлять символы
        if len(cur_state) < len(self.target_string):
            # Все буквы русского алфавита в нижнем и верхнем регистре, а также пробел
            alphabets = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
            return list(alphabets + ' ' + alphabets.upper())
        else:
            # Если достигнута длина целевой строки, больше действий нет
            return []

    def result(self, cur_state, action):
        """
        Применяет действие (добавляет символ) к текущему состоянию
        и возвращает новое состояние.
        """
        return cur_state + action

    def is_goal(self, cur_state):
        """Проверяет, является ли текущее состояние целевым"""
        return cur_state == self.target_string

    def heuristic(self, cur_state):
        """
        Эвристическая функция для оценки расстояния от текущего состояния до цели.
        Используется в жадном алгоритме для выбора следующего состояния.
        """
        # Вычисляем количество несовпадающих символов на соответствующих позициях
        # (только для той части, где строки перекрываются)
        dist = sum([1 if cur_state[i] != self.target_string[i] else 0
                    for i in range(len(cur_state))])

        # Разница в длине между целевой и текущей строкой
        diff = len(self.target_string) - len(cur_state)

        # Общая оценка: сумма несовпадений и недостающих символов
        return dist + diff 

if __name__ == '__main__':
    # Парсинг аргументов командной строки
    args = build_arg_parser().parse_args()

    # Создание экземпляра задачи
    problem = CustomProblem()

    # Настройка задачи: установка целевой строки и начального состояния
    problem.set_target(args.input_string)
    problem.initial_state = args.initial_state

    # Запуск жадного поиска (greedy search) для решения задачи
    output = ss.greedy(problem)

    # Вывод результатов
    print('\nЦелевая строка:', args.input_string)
    print('\nПуть к решению:')
    
    # Вывод каждого шага в пути решения
    for item in output.path():
        print(item)

## Решение задачи о раскраске областей

In [ ]:
from simpleai.search import CspProblem, backtrack  # Импорт классов для решения задач удовлетворения ограничений

def constraint_func(names, values):
    """Функция ограничения: проверяет, что значения двух переменных не равны"""
    return values[0] != values[1]  # Цвета двух людей должны быть разными

if __name__ == '__main__':
    # Определение переменных - имен людей, которым нужно назначить цвета
    names = ('Марк', 'Джулия', 'Стив', 'Аманда', 'Брайан', 
             'Джоанн', 'Дерек', 'Аллан', 'Мишель', 'Келли')
    
    # Создание доменов (возможных значений) для каждой переменной
    # Каждый человек может быть покрашен в один из 4 цветов: красный, зеленый, синий или серый
    colors = dict((name, ['красный', 'зелёный', 'синий', 'серый']) for name in names)

    # Определение ограничений задачи
    # Каждое ограничение представляет собой пару людей, которые не могут иметь одинаковый цвет
    constraints = [
        # Марк не может иметь тот же цвет, что и Джулия и Стив
        (('Марк', 'Джулия'), constraint_func),
        (('Марк', 'Стив'), constraint_func),
        
        # Джулия имеет много связей - не может иметь тот же цвет, что и несколько людей
        (('Джулия', 'Стив'), constraint_func),
        (('Джулия', 'Аманда'), constraint_func),
        (('Джулия', 'Дерек'), constraint_func),
        (('Джулия', 'Брайан'), constraint_func),
        
        # Ограничения для Стива
        (('Стив', 'Аманда'), constraint_func),
        (('Стив', 'Аллан'), constraint_func),
        (('Стив', 'Мишель'), constraint_func),
        
        # Ограничения для Аманды
        (('Аманда', 'Мишель'), constraint_func),
        (('Аманда', 'Джоанн'), constraint_func),
        (('Аманда', 'Дерек'), constraint_func),
        
        # Ограничения для Брайана
        (('Брайан', 'Дерек'), constraint_func),
        (('Брайан', 'Келли'), constraint_func),
        
        # Ограничения для Джоанн
        (('Джоанн', 'Мишель'), constraint_func),
        (('Джоанн', 'Аманда'), constraint_func),  # Дублирует предыдущее ограничение Аманда-Джоанн
        (('Джоанн', 'Дерек'), constraint_func),
        (('Джоанн', 'Келли'), constraint_func),
        
        # Ограничения для Дерека
        (('Дерек', 'Келли'), constraint_func),
    ]

    # Создание объекта задачи удовлетворения ограничений (CSP)
    problem = CspProblem(names, colors, constraints)

    # Решение задачи с помощью алгоритма обратного отслеживания (backtracking)
    output = backtrack(problem)
    
    # Вывод результатов
    print('\nРаспределение цветов:\n')
    for k, v in output.items():
        print(k, '==>', v)

## Создание головоломки 8

In [ ]:
from simpleai.search import astar, SearchProblem  # Импорт алгоритма A* и базового класса для задач поиска
    
class PuzzleSolver(SearchProblem):
    """Класс для решения головоломки 'Пятнашки' (3x3) с использованием алгоритма A*"""
    
    def actions(self, cur_state):
        """
        Возвращает возможные действия (ходы) из текущего состояния.
        Действие - это число, которое можно передвинуть на пустую клетку ('e').
        """
        rows = string_to_list(cur_state)  # Преобразуем строку состояния в список строк
        row_empty, col_empty = get_location(rows, 'e')  # Находим координаты пустой клетки

        actions = []
        # Проверяем возможность перемещения чисел в пустую клетку со всех четырех сторон
        if row_empty > 0:  # Можно переместить число сверху
            actions.append(rows[row_empty - 1][col_empty])
        if row_empty < 2:  # Можно переместить число снизу
            actions.append(rows[row_empty + 1][col_empty])
        if col_empty > 0:  # Можно переместить число слева
            actions.append(rows[row_empty][col_empty - 1])
        if col_empty < 2:  # Можно переместить число справа
            actions.append(rows[row_empty][col_empty + 1])

        return actions

    def result(self, state, action):
        """
        Применяет действие (перемещает число) к текущему состоянию
        и возвращает новое состояние.
        """
        rows = string_to_list(state)  # Преобразуем строку в список
        row_empty, col_empty = get_location(rows, 'e')  # Находим пустую клетку
        row_new, col_new = get_location(rows, action)  # Находим клетку с числом для перемещения

        # Меняем местами пустую клетку и число
        rows[row_empty][col_empty], rows[row_new][col_new] = \
                rows[row_new][col_new], rows[row_empty][col_empty]

        return list_to_string(rows)  # Возвращаем новое состояние в строковом формате

    def is_goal(self, state):
        """Проверяет, является ли текущее состояние целевым"""
        return state == GOAL

    def heuristic(self, state):
        """
        Эвристическая функция для алгоритма A*.
        Вычисляет манхэттенское расстояние - сумму расстояний каждой плитки
        от ее текущей позиции до целевой позиции.
        """
        rows = string_to_list(state)  # Преобразуем строку в список

        distance = 0  # Суммарное расстояние

        # Для каждой плитки (1-8) и пустой клетки ('e')
        for number in '12345678e':
            row_new, col_new = get_location(rows, number)  # Текущая позиция плитки
            row_new_goal, col_new_goal = goal_positions[number]  # Целевая позиция плитки

            # Суммируем манхэттенское расстояние (разница по строкам + разница по столбцам)
            distance += abs(row_new - row_new_goal) + abs(col_new - col_new_goal)

        return distance

# Вспомогательные функции для преобразования между форматами

def list_to_string(input_list):
    """Преобразует список списков (матрицу 3x3) в строку с разделителями"""
    return '\n'.join(['-'.join(x) for x in input_list])

def string_to_list(input_string):
    """Преобразует строку с разделителями в список списков (матрицу 3x3)"""
    return [x.split('-') for x in input_string.split('\n')]
 
def get_location(rows, input_element):
    """Находит координаты элемента в матрице"""
    for i, row in enumerate(rows):
        for j, item in enumerate(row):
            if item == input_element:
                return i, j  # Возвращаем (строка, столбец)

# Определение целевого и начального состояний головоломки

GOAL = '''1-2-3
4-5-6
7-8-e'''  # Целевое состояние (решённая головоломка)

INITIAL = '''1-e-2
6-3-4
7-5-8'''  # Начальное состояние (перемешанная головоломка)

# Создание словаря целевых позиций для каждой плитки
# Необходимо для быстрого вычисления эвристики

goal_positions = {}  # Словарь: плитка -> (целевая_строка, целевой_столбец)
rows_goal = string_to_list(GOAL)  # Преобразуем цель в список

# Заполняем словарь целевыми позициями для каждой плитки (1-8) и пустой клетки ('e')
for number in '12345678e':
    goal_positions[number] = get_location(rows_goal, number)

# Запускаем алгоритм A* для решения головоломки
result = astar(PuzzleSolver(INITIAL))

# Выводим путь решения
for i, (action, state) in enumerate(result.path()):
    print()  # Пустая строка для разделения шагов
    
    # Выводим описание действия в зависимости от позиции в пути
    if action == None:
        print('Начальная конфигурация')
    elif i == len(result.path()) - 1:
        print('После передвижения', action, 'в пустую клетку. Цель достигнута!')
    else:
        print('После передвижения', action, 'в пустую клетку')

    print(state)  # Выводим текущее состояние головоломки

## Создание решателя для прохождения лабиринта

In [ ]:
import math
from simpleai.search import SearchProblem, astar

class MazeSolver(SearchProblem):
    """Класс для решения задачи поиска пути в лабиринте с помощью алгоритма A*"""
    
    def __init__(self, board):
        """
        Инициализация решателя лабиринта.
        
        Args:
            board: Двумерный список (матрица) символов, представляющий лабиринт
        """
        self.board = board  # Карта лабиринта
        self.goal = (0, 0)  # Целевая позиция (по умолчанию)

        # Поиск стартовой ('o') и целевой ('x') позиций на карте
        for y in range(len(self.board)):
            for x in range(len(self.board[y])):
                if self.board[y][x].lower() == "o":
                    self.initial = (x, y)  # Начальная позиция (координаты x, y)
                elif self.board[y][x].lower() == "x":
                    self.goal = (x, y)  # Целевая позиция

        # Инициализация родительского класса с начальным состоянием
        super(MazeSolver, self).__init__(initial_state=self.initial)

    def actions(self, state):
        """
        Возвращает список возможных действий из текущего состояния.
        Действие - это перемещение в соседнюю клетку.
        
        Args:
            state: Текущая позиция (x, y)
            
        Returns:
            Список разрешенных направлений движения
        """
        actions = []
        # Проверяем все возможные направления движения из словаря COSTS
        for action in COSTS.keys():
            newx, newy = self.result(state, action)  # Вычисляем новую позицию
            # Если новая позиция не является стеной ('#'), добавляем действие
            if self.board[newy][newx] != "#":
                actions.append(action)

        return actions

    def result(self, state, action):
        """
        Применяет действие (перемещение) к текущему состоянию.
        
        Args:
            state: Текущая позиция (x, y)
            action: Направление движения (например, "up", "down left")
            
        Returns:
            Новая позиция после выполнения действия
        """
        x, y = state  # Распаковываем текущие координаты

        # Обновляем координаты в зависимости от направления движения
        if action.count("up"):
            y -= 1  # Движение вверх уменьшает y
        if action.count("down"):
            y += 1  # Движение вниз увеличивает y
        if action.count("left"):
            x -= 1  # Движение влево уменьшает x
        if action.count("right"):
            x += 1  # Движение вправо увеличивает x

        new_state = (x, y)  # Формируем новое состояние

        return new_state

    def is_goal(self, state):
        """
        Проверяет, достигнута ли целевая позиция.
        
        Args:
            state: Текущая позиция (x, y)
            
        Returns:
            True, если текущая позиция совпадает с целевой, иначе False
        """
        return state == self.goal

    def cost(self, state, action, state2):
        """
        Возвращает стоимость выполнения действия.
        В данном случае используется фиксированная стоимость из словаря COSTS.
        
        Args:
            state: Начальное состояние
            action: Выполняемое действие
            state2: Конечное состояние
            
        Returns:
            Стоимость перемещения
        """
        return COSTS[action]

    def heuristic(self, state):
        """
        Эвристическая функция для алгоритма A*.
        Использует евклидово расстояние до цели.
        
        Args:
            state: Текущая позиция (x, y)
            
        Returns:
            Евклидово расстояние до цели
        """
        x, y = state  # Текущие координаты
        gx, gy = self.goal  # Целевые координаты

        # Вычисляем евклидово расстояние по формуле sqrt((x2-x1)² + (y2-y1)²)
        return math.sqrt((x - gx) ** 2 + (y - gy) ** 2)

if __name__ == "__main__":
    # Определение карты лабиринта в виде строки
    MAP = """
    ##############################
    #         #              #   #
    # ####    ########       #   #
    #  o #    #              #   #
    #    ###     #####  ######   #
    #      #   ###   #           #
    #      #     #   #  #  #   ###
    #     #####    #    #  # x   #
    #              #       #     #
    ##############################
    """

    # Вывод исходной карты лабиринта
    print(MAP)
    # Преобразование строки карты в двумерный список символов
    # split("\n") разбивает по строкам, if x отфильтровывает пустые строки
    MAP = [list(x) for x in MAP.split("\n") if x]

    # Определение стоимостей перемещений
    cost_regular = 1.0  # Стоимость обычного перемещения (по вертикали/горизонтали)
    cost_diagonal = 1.7  # Стоимость диагонального перемещения (приблизительно √2)

    # Словарь стоимостей для всех возможных направлений движения
    COSTS = {
        "up": cost_regular,           # Вверх
        "down": cost_regular,         # Вниз
        "left": cost_regular,         # Влево
        "right": cost_regular,        # Вправо
        "up left": cost_diagonal,     # Вверх-влево (диагональ)
        "up right": cost_diagonal,    # Вверх-вправо (диагональ)
        "down left": cost_diagonal,   # Вниз-влево (диагональ)
        "down right": cost_diagonal,  # Вниз-вправо (диагональ)
    }

    # Создание экземпляра решателя лабиринта
    problem = MazeSolver(MAP)

    # Запуск алгоритма A* для поиска пути
    # graph_search=True означает, что алгоритм будет отслеживать посещенные состояния
    result = astar(problem, graph_search=True)

    # Извлечение пути из результата (список состояний, пройденных до цели)
    path = [x[1] for x in result.path()]

    # Вывод карты лабиринта с найденным путем
    print()
    for y in range(len(MAP)):
        for x in range(len(MAP[y])):
            if (x, y) == problem.initial:  # Начальная позиция
                print('o', end='')
            elif (x, y) == problem.goal:   # Целевая позиция
                print('x', end='')
            elif (x, y) in path:           # Ячейки пути
                print('·', end='')         # Символ точки для отображения пути
            else:                          # Остальные ячейки
                print(MAP[y][x], end='')   # Оригинальный символ карты
        print()  # Переход на новую строку после каждой строки карты